In [1]:
from pydataset import data
import pyspark
import pyspark.ml
from pyspark.sql.functions import *

spark = pyspark.sql.SparkSession.builder.getOrCreate()

df = spark.createDataFrame(data('tips'))

In [54]:
dir(pyspark.ml.feature)

['Binarizer',
 'BucketedRandomProjectionLSH',
 'BucketedRandomProjectionLSHModel',
 'Bucketizer',
 'ChiSqSelector',
 'ChiSqSelectorModel',
 'CountVectorizer',
 'CountVectorizerModel',
 'DCT',
 'DecisionTreeParams',
 'ElementwiseProduct',
 'FeatureHasher',
 'HasAggregationDepth',
 'HasCheckpointInterval',
 'HasCollectSubModels',
 'HasDistanceMeasure',
 'HasElasticNetParam',
 'HasFeaturesCol',
 'HasFitIntercept',
 'HasHandleInvalid',
 'HasInputCol',
 'HasInputCols',
 'HasLabelCol',
 'HasLoss',
 'HasMaxIter',
 'HasNumFeatures',
 'HasOutputCol',
 'HasOutputCols',
 'HasParallelism',
 'HasPredictionCol',
 'HasProbabilityCol',
 'HasRawPredictionCol',
 'HasRegParam',
 'HasSeed',
 'HasSolver',
 'HasStandardization',
 'HasStepSize',
 'HasThreshold',
 'HasThresholds',
 'HasTol',
 'HasVarianceCol',
 'HasWeightCol',
 'HashingTF',
 'IDF',
 'IDFModel',
 'Imputer',
 'ImputerModel',
 'IndexToString',
 'JavaEstimator',
 'JavaMLReadable',
 'JavaMLWritable',
 'JavaModel',
 'JavaParams',
 'JavaTransformer'

In [2]:
train, test = df.randomSplit([0.8, 0.2], seed=123)
# train, validate, test = df.randomSplit([0.6, 0.2, 0.2])

In [3]:
print('train shape')
train.count(), len(train.columns)

train shape


(190, 7)

In [4]:
print('test shape')
test.count(), len(test.columns)

test shape


(54, 7)

In [5]:
def shape(df: pyspark.sql.DataFrame):
    return df.count(), len(df.columns)

In [6]:
shape(train)

(190, 7)

## Regression

In [7]:
train.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     12.69| 2.0|  Male|    No|Sat|Dinner|   2|
|     13.37| 2.0|  Male|    No|Sat|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     16.29|3.71|  Male|    No|Sun|Dinner|   3|
|     16.97| 3.5|Female|    No|Sun|Dinner|   3|
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     17.81|2.34|  Male|    No|Sat|Dinner|   4|
|     17.92|4.08|  Male|    No|Sat|Dinner|   2|
|     19.65| 3.0|Female|    No|Sat|Dinner|   2|
|     19.82|3.18|  Male|    No|Sat|Dinner|   2|
|     20.29|2.75|Female|    No|Sat|Dinner|   2|
|     20.65|3.35|  Male|    No|Sat|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|      21.7| 4.3|  Male|    No|Sat|Dinne

- `tip ~ total_bill`: predict tip based on total bill
- `tip ~ total_bill + size`: predict tip based on total bill and size
- `tip ~ .`: predict tip based on all the other features in the dataset

In [18]:
# nb: spark's rformula does encoding
rf = pyspark.ml.feature.RFormula(formula="tip ~ total_bill + size").fit(train)

rf.transform(train).show()

+----------+----+------+------+---+------+----+-----------+-----+
|total_bill| tip|   sex|smoker|day|  time|size|   features|label|
+----------+----+------+------+---+------+----+-----------+-----+
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2| [8.77,2.0]|  2.0|
|     12.69| 2.0|  Male|    No|Sat|Dinner|   2|[12.69,2.0]|  2.0|
|     13.37| 2.0|  Male|    No|Sat|Dinner|   2|[13.37,2.0]|  2.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|[14.78,2.0]| 3.23|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|[14.83,2.0]| 3.02|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|[15.42,2.0]| 1.57|
|     16.29|3.71|  Male|    No|Sun|Dinner|   3|[16.29,3.0]| 3.71|
|     16.97| 3.5|Female|    No|Sun|Dinner|   3|[16.97,3.0]|  3.5|
|     16.99|1.01|Female|    No|Sun|Dinner|   2|[16.99,2.0]| 1.01|
|     17.81|2.34|  Male|    No|Sat|Dinner|   4|[17.81,4.0]| 2.34|
|     17.92|4.08|  Male|    No|Sat|Dinner|   2|[17.92,2.0]| 4.08|
|     19.65| 3.0|Female|    No|Sat|Dinner|   2|[19.65,2.0]|  3.0|
|     19.8

In [9]:
train_input = rf.transform(train).select('features', 'label')
train_input.show()

+-----------+-----+
|   features|label|
+-----------+-----+
| [8.77,2.0]|  2.0|
|[12.69,2.0]|  2.0|
|[13.37,2.0]|  2.0|
|[14.78,2.0]| 3.23|
|[14.83,2.0]| 3.02|
|[15.42,2.0]| 1.57|
|[16.29,3.0]| 3.71|
|[16.97,3.0]|  3.5|
|[16.99,2.0]| 1.01|
|[17.81,4.0]| 2.34|
|[17.92,2.0]| 4.08|
|[19.65,2.0]|  3.0|
|[19.82,2.0]| 3.18|
|[20.29,2.0]| 2.75|
|[20.65,3.0]| 3.35|
|[21.01,3.0]|  3.5|
|[21.58,2.0]| 3.92|
| [21.7,2.0]|  4.3|
|[24.59,4.0]| 3.61|
|[25.29,4.0]| 4.71|
+-----------+-----+
only showing top 20 rows



In [20]:
lr = pyspark.ml.regression.LinearRegression()
lr

LinearRegression_ca53afdfd93d

In [21]:
# print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
regParam: regularization parameter (>= 0). (default: 0.0)
solver: The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (default: auto)
standardization: whether to standardize the training features before fitting the model.

In [23]:
lr_fit = lr.fit(train_input)
lr_fit.transform(train_input).show()

+-----------+-----+------------------+
|   features|label|        prediction|
+-----------+-----+------------------+
| [8.77,2.0]|  2.0|1.9717337092055747|
|[12.69,2.0]|  2.0|  2.28547899247938|
|[13.37,2.0]|  2.0| 2.339904194679938|
|[14.78,2.0]| 3.23|2.4527564521840364|
|[14.83,2.0]| 3.02| 2.456758305287019|
|[15.42,2.0]| 1.57| 2.503980171902209|
|[16.29,3.0]| 3.71|2.8367765178831013|
|[16.97,3.0]|  3.5| 2.891201720083659|
|[16.99,2.0]| 1.01|2.6296383593358503|
|[17.81,4.0]| 2.34|3.2215969542027625|
|[17.92,2.0]| 4.08|2.7040728270513195|
|[19.65,2.0]|  3.0| 2.842536944414504|
|[19.82,2.0]| 3.18|2.8561432449646436|
|[20.29,2.0]| 2.75|2.8937606641326763|
|[20.65,3.0]| 3.35|  3.18573810846315|
|[21.01,3.0]|  3.5|3.2145514508046222|
|[21.58,2.0]| 3.92|2.9970084741896175|
| [21.7,2.0]|  4.3|3.0066129216367745|
|[24.59,4.0]| 3.61| 3.764248234967151|
|[25.29,4.0]| 4.71|3.8202741784089014|
+-----------+-----+------------------+
only showing top 20 rows



In [24]:
lr_fit.summary

In [26]:
dir(lr_fit.summary)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_call_java',
 '_create_from_java_class',
 '_java_obj',
 '_new_java_array',
 '_new_java_obj',
 'coefficientStandardErrors',
 'degreesOfFreedom',
 'devianceResiduals',
 'explainedVariance',
 'featuresCol',
 'labelCol',
 'meanAbsoluteError',
 'meanSquaredError',
 'numInstances',
 'objectiveHistory',
 'pValues',
 'predictionCol',
 'predictions',
 'r2',
 'r2adj',
 'residuals',
 'rootMeanSquaredError',
 'tValues',
 'totalIterations']

In [25]:
lr_fit.summary.r2, lr_fit.summary.rootMeanSquaredError

(0.44244602311951, 0.9975913515122248)

How do we do on the test data?

In [27]:
test_input = rf.transform(test)
lr_fit.transform(test_input).show(4)

+----------+----+------+------+---+------+----+-----------+-----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|   features|label|        prediction|
+----------+----+------+------+---+------+----+-----------+-----+------------------+
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|[10.27,2.0]| 1.71| 2.091789302295041|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|[10.33,3.0]| 1.67|2.3597556280076217|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|[10.34,3.0]| 1.66| 2.360555998628218|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|[15.04,2.0]| 1.96| 2.473566088319544|
+----------+----+------+------+---+------+----+-----------+-----+------------------+
only showing top 4 rows



In [28]:
evaluator = pyspark.ml.evaluation.RegressionEvaluator()
rmse = evaluator.evaluate(lr_fit.transform(test_input))
'{:.4}'.format(rmse)

'1.058'

## Classification

In [47]:
rf = pyspark.ml.feature.RFormula(formula='time ~ total_bill + size').fit(train)
train_input = rf.transform(train)
train_input.show(50)

+----------+----+------+------+----+------+----+-----------+-----+
|total_bill| tip|   sex|smoker| day|  time|size|   features|label|
+----------+----+------+------+----+------+----+-----------+-----+
|      8.77| 2.0|  Male|    No| Sun|Dinner|   2| [8.77,2.0]|  0.0|
|     12.69| 2.0|  Male|    No| Sat|Dinner|   2|[12.69,2.0]|  0.0|
|     13.37| 2.0|  Male|    No| Sat|Dinner|   2|[13.37,2.0]|  0.0|
|     14.78|3.23|  Male|    No| Sun|Dinner|   2|[14.78,2.0]|  0.0|
|     14.83|3.02|Female|    No| Sun|Dinner|   2|[14.83,2.0]|  0.0|
|     15.42|1.57|  Male|    No| Sun|Dinner|   2|[15.42,2.0]|  0.0|
|     16.29|3.71|  Male|    No| Sun|Dinner|   3|[16.29,3.0]|  0.0|
|     16.97| 3.5|Female|    No| Sun|Dinner|   3|[16.97,3.0]|  0.0|
|     16.99|1.01|Female|    No| Sun|Dinner|   2|[16.99,2.0]|  0.0|
|     17.81|2.34|  Male|    No| Sat|Dinner|   4|[17.81,4.0]|  0.0|
|     17.92|4.08|  Male|    No| Sat|Dinner|   2|[17.92,2.0]|  0.0|
|     19.65| 3.0|Female|    No| Sat|Dinner|   2|[19.65,2.0]|  

In [38]:
lr = pyspark.ml.classification.LogisticRegression()
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal wi

In [36]:
lr_fit = lr.fit(train_input)

In [35]:
dir(lr_fit.summary)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_call_java',
 '_create_from_java_class',
 '_java_obj',
 '_new_java_array',
 '_new_java_obj',
 'accuracy',
 'areaUnderROC',
 'fMeasureByLabel',
 'fMeasureByThreshold',
 'falsePositiveRateByLabel',
 'featuresCol',
 'labelCol',
 'labels',
 'objectiveHistory',
 'pr',
 'precisionByLabel',
 'precisionByThreshold',
 'predictionCol',
 'predictions',
 'probabilityCol',
 'recallByLabel',
 'recallByThreshold',
 'roc',
 'totalIterations',
 'truePositiveRateByLabel',
 'weightedFMeasure',
 'weightedFalsePositiveRate',
 'weightedPrecision',
 'weightedRecall',
 'weightedTruePositiveRate']

In [40]:
# area under TPR (recall) vs FPR (FP / (FP + TN)) curve
# https://en.wikipedia.org/wiki/Receiver_operating_characteristic
lr_fit.summary.areaUnderROC

0.6526599326599327

In [39]:
evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator()
test_auc = evaluator.evaluate(lr_fit.transform(rf.transform(test)))
test_auc

0.5234521575984991

In [51]:
test_input = rf.transform(test)

In [53]:
(lr_fit.transform(test_input)
 .select('time', 'total_bill', 'size', 'label', 'probability', 'prediction')
 .groupby('prediction') # predicted == rows
 .pivot('label') # actual values are columns
 .count()
 .show())

+----------+---+----+
|prediction|0.0| 1.0|
+----------+---+----+
|       0.0| 40|  13|
|       1.0|  1|null|
+----------+---+----+

